In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

tf.__version__

# 케라스의 순환 층 : SimpleRNN

- 시퀀스 배치를 처리 : (batch_size, timesteps, input_features) 크기의 입력을 받음
- 두 가지 모드로 실행: return_sequences로 선택
  - 각 타임스텝의 출력을 모은 전체 시퀀스를 반환: 크기가 (batch_size, timesteps, output_features) 3D 텐서, : return_sequences=True
  - 입력 시퀀스에 대한 마지막 출력만 반환 : 크기가 (batch_size, output_features)인 2D 텐서


### 예제1. SimpleRNN을 사용하여 마지막 타임스텝의 출력 얻기

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN

model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32))

model.summary()

### 예제2. SimpleRNN을 사용하여 전체 상태 시퀀스 반환

In [ ]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))

model.summary()

### 예제3. 여러 개의 순환층을 사용한 RNN 모델
- 중간층들이 전체 출력 시퀀스를 반환하도록 설정해야 함

In [ ]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))

model.summary()

## IMDB 영화리뷰 분류를 위한 RNN : SimpleRNN 층 사용

### 1) imdb 데이터 로딩 및 데이터 전처리


In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

max_features = 10000
maxlen = 500
batch_size = 32

print('데이터 로딩...')
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words = max_features)
print(len(input_train), '훈련 시퀀스')
print(len(input_test), '테스트 시퀀스')

print('시퀀스 패딩(samples x time)')
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print('input_train 크기:', input_train.shape)
print('input_test 크기:', input_test.shape)

### 2) IMDB 분류를 위한 간단한 순환 신경망 정의 및 컴파일

In [ ]:
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

### 3) 순환신경망 훈련

In [ ]:
history = model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

In [ ]:
from tensorflow.keras import callbacks, optimizers

rmsprop = optimizers.RMSprop(learning_rate=1e-4)
model.compile(optimizer=rmsprop, loss='binary_crossentropy', 
              metrics=['accuracy'])

checkpoint_cb = callbacks.ModelCheckpoint('model/best-simplernn-model2.h5')
early_stopping_cb = callbacks.EarlyStopping(patience=3,
                                            restore_best_weights=True)
history = model.fit(input_train, y_train,
                    epochs=50,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=[checkpoint_cb, early_stopping_cb])

### 4)  순환 신경망 모델의 성능(훈련, 검증의 손실과 정확도) 시각화

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc)+1)

plt.figure(figsize=(9,6))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and Validation accuracy')
plt.grid()
plt.legend()
plt.show()

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss)+1)

plt.figure(figsize=(9,6))
plt.plot(epochs, loss, 'ro', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.grid()
plt.legend()
plt.show()

#### 모델 테스트

In [ ]:
rnn_model = keras.models.load_model('model/best-simplernn-model2.h5')

rnn_model.evaluate(input_test, y_test)